In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from datetime import datetime, timedelta
import selenium.webdriver.common.keys
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
# Install the ChromeDriver executable and start a Chrome browser using Selenium
driver = webdriver.Chrome()


In [3]:
# Navigate to the webpage
driver.get('https://merolagani.com/Floorsheet.aspx')


In [4]:
# Define a function to extract data from the current page
def extract_page_data():
    # Let's use BeautifulSoup to parse the page source
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Extract the data you need from the current page
    # Adjust the selectors based on your HTML structure
    data = []
    rows = soup.select('.table-bordered tbody tr')
    for row in rows:
        columns = row.find_all('td')
        row_data = [column.get_text(strip=True) for column in columns]
        data.append(row_data)
    
    return data


In [5]:
# Assuming the "Items per page" filter is a dropdown with the class 'ng-untouched'
# Wait for the dropdown to be present on the page
#wait = WebDriverWait(driver, 10)
#items_per_page_dropdown = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'ng-untouched')))
#items_per_page_dropdown.click()
# Find the option with value '500' and click on it
#option_500 = wait.until(EC.element_to_be_clickable((By.XPATH, '//option[@value="500"]')))
#option_500.click()
#filter_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'box__filter--search')))
#filter_button.click()
#print(driver.page_source)
#driver.implicitly_wait(2)


In [5]:
# Calculate the specific date (e.g., start from one week ago)
start_date_str = '01/18/2024'
start_date = datetime.strptime(start_date_str , '%m/%d/%Y')
date_array = []


# Get the current date
end_date = datetime.now()

# Iterate through the date range
while start_date <= end_date:
    # Format the current date as MM/DD/YYYY and print
    current_date_str = start_date.strftime('%m/%d/%Y')
#     date_input.clear()
#     date_input.send.keys(current_date_str)
    date_array.append(  current_date_str )
     
    # Move to the next date
    start_date += timedelta(days=1)
print(date_array)

['01/18/2024', '01/19/2024', '01/20/2024', '01/21/2024']


In [6]:
# Define a function to click the next page button
def click_next_page():
    try:
        # Find the next page element and click it
        next_page = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//a[text()="Next"]'))
        )
        next_page.click()
        return True
    except Exception as e:
        print(f"Error clicking next page: {e}")
        return False


In [7]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Find the input element by its ID
#input_element = driver.find_element_by_id('ctl00_ContentPlaceHolder1_txtFloorsheetDateFilter')
# Initialize an empty list to store the data from all pages
all_data = []
for date_str in date_array:
    try:

        print(f"getting data from {date_str}")
        # Find the input element by its ID
        input_element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, 'ctl00_ContentPlaceHolder1_txtFloorsheetDateFilter')))
        input_element.clear()
        input_element.send_keys(date_str)
       
        # Print the extracted value
        #print("Date Value:", date_value)
        # Press Enter to confirm the new date (optional, depends on the website's behavior)
        input_element.send_keys(Keys.ENTER)
    
        # Wait for the element to be clickable (you might need to adjust the timeout)
        search_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'ctl00_ContentPlaceHolder1_lbtnSearchFloorsheet')))
        # Click the "Search" button
        search_button.click()
        # Extract the total pages value from the content
        #total_pages_text = span_element.get_text(strip=True)
        #start_index = total_pages_text.find("[Total pages:") + len("[Total pages:")
        #end_index = total_pages_text.find("]", start_index)

        #total_pages_value = total_pages_text[start_index:end_index]
        #num_pages_to_scrape = int(total_pages_value)

        #testing number of pages:
         # Find the span element by its ID
        date_span = driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_marketDate')
        # Extract the text value from the span element
        date_value = date_span.text
        
        num_pages_to_scrape = 2
        # Loop through pages
        page_num = 0
        for page_num in range(num_pages_to_scrape):
            print(f"Scraping data from page {page_num + 1}")

            # Extract data from the current page
            current_data = extract_page_data()
            all_data.extend(current_data)

            # Click the next page
            if not click_next_page():
                print("No more pages to scrape.")
                break

    except Exception as e:
        print (f"Error printing the page+date_str+{e}")
   

getting data from 01/18/2024
Scraping data from page 1
Scraping data from page 2
getting data from 01/19/2024
Error printing the page+date_str+Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="ctl00_ContentPlaceHolder1_marketDate"]"}
  (Session info: chrome=120.0.6099.234); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x00000001012584dc chromedriver + 4162780
1   chromedriver                        0x0000000101250664 chromedriver + 4130404
2   chromedriver                        0x0000000100ea7bc0 chromedriver + 293824
3   chromedriver                        0x0000000100eee040 chromedriver + 581696
4   chromedriver                        0x0000000100f2dab0 chromedriver + 842416
5   chromedriver                        0x0000000100ee16bc chromedriver + 530108
6   chromedriver                    

In [8]:

# Set the number of pages you want to scrape
span_element = soup.find('span', id='ctl00_ContentPlaceHolder1_PagerControl1_litRecords')


In [9]:
# Close the browser
driver.quit()

In [10]:
# Print the collected data
print("Collected Data:")
for row in all_data:
    print(row)

Collected Data:
['1', '2024011805160809', 'BBC', '45', '22', '10', '3,830.00', '38,300.00']
['2', '2024011805160827', 'CHDC', '58', '49', '25', '1,155.60', '28,890.00']
['3', '2024011805160826', 'CHDC', '58', '50', '10', '1,155.00', '11,550.00']
['4', '2024011805160825', 'CHDC', '58', '42', '100', '1,155.00', '115,500.00']
['5', '2024011805160824', 'CHDC', '58', '42', '50', '1,155.00', '57,750.00']
['6', '2024011805160823', 'CHDC', '58', '34', '100', '1,133.00', '113,300.00']
['7', '2024011805160822', 'CHDC', '58', '28', '200', '1,133.00', '226,600.00']
['8', '2024011805160821', 'CHDC', '58', '37', '15', '1,133.00', '16,995.00']
['9', '2024011805160676', 'CHDC', '58', '37', '15', '1,133.00', '16,995.00']
['10', '2024011805160675', 'CHDC', '58', '42', '85', '1,133.00', '96,305.00']
['11', '2024011805160588', 'CHDC', '58', '42', '15', '1,133.00', '16,995.00']
['12', '2024011805160587', 'CHDC', '58', '45', '85', '1,100.00', '93,500.00']
['13', '2024011805160731', 'CIT', '38', '40', '20', 